<a href="https://colab.research.google.com/github/ericbonelli/Cientista-de-Dados_EBAC/blob/main/AdaBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚡  AdaBoost

## ✅ 1. O que é o AdaBoost?

**AdaBoost** (Adaptive Boosting) é um algoritmo de aprendizado de máquina que combina vários modelos fracos (geralmente árvores simples, chamadas "stumps") para formar um modelo forte e mais preciso.

A ideia central é **focar nos exemplos que o modelo anterior errou**, ajustando os pesos dos dados a cada rodada de treinamento. Isso permite que o modelo aprenda progressivamente com seus próprios erros.

Ele funciona bem com classificadores fracos e é muito usado por sua simplicidade e boa performance — desde que os dados não tenham muito **ruído ou outliers**, já que o AdaBoost é sensível a isso.

---


## ✅ 2. Passo a passo do algoritmo AdaBoost

1. **Inicialização dos pesos**  
   - Todos os exemplos do dataset começam com o mesmo peso.

2. **Treinamento sequencial de modelos fracos**  
   - Treina-se um modelo (ex: árvore simples).
   - Calcula-se o erro ponderado (considerando os pesos dos exemplos).

3. **Ajuste dos pesos dos exemplos**  
   - Exemplos que o modelo errou recebem mais peso (para ganhar mais atenção no próximo modelo).

4. **Cálculo do peso do modelo**  
   - Modelos com menor erro ganham mais peso na combinação final.

5. **Repetição**  
   - Repete-se o processo para `n_estimators` vezes.

6. **Agregação das previsões**  
   - A previsão final é feita com base em uma votação ponderada dos modelos.

---


## ✅ 3. Cinco diferenças entre Random Forest e AdaBoost

| Característica           | **Random Forest**                              | **AdaBoost**                                      |
|--------------------------|-------------------------------------------------|--------------------------------------------------|
| Estratégia de ensemble   | Paralela (várias árvores ao mesmo tempo)       | Sequencial (um modelo por vez)                   |
| Peso dos modelos         | Todos os modelos têm o mesmo peso              | Modelos com menor erro recebem maior peso        |
| Peso nos exemplos        | Não altera                                     | Aumenta peso dos erros para focar nos difíceis   |
| Tolerância a ruído       | Alta (robusto a outliers)                      | Baixa (sensível a outliers)                      |
| Complexidade             | Mais simples de ajustar                        | Requer mais cuidado (learning rate, overfitting) |

---

## ✅ 4. Implementação em Python

Vamos usar o conjunto de dados Iris como exemplo e aplicar o AdaBoost, sendo demonstrando passo a passo de forma manual e depois usando scikit-learn

---

# ⚡ 4.1. AdaBoost com Dataset Iris - Manual

Vamos entender o funcionamento do AdaBoost recriando suas 6 etapas principais com o dataset Iris.

---

## 📌 Etapa 1: - Inicialização dos pesos

Todos os exemplos do dataset começam com o mesmo peso.
Neste exemplo didático, usaremos apenas duas classes e duas features para simplificar.


In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

# Carregar o dataset com duas classes e duas features
iris = load_iris()
X_full = iris.data[iris.target != 2][:, :2]  # Apenas classes 0 e 1, duas features
y_full = iris.target[iris.target != 2]

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=42)

# Inicializar pesos uniformemente (apenas no treino)
n_amostras = len(y_train)
pesos = np.ones(n_amostras) / n_amostras

## 📌 Etapa 2, 3 e 4 - Treinamento sequencial de modelos fracos ajustes dos pesos e cálculo dos próximos pesos

Treinamos um modelo fraco (stump) e calculamos o erro ponderado.

In [10]:
from sklearn.tree import DecisionTreeClassifier

n_estimators = 5
modelos = []
alfas = []

for i in range(n_estimators):
    modelo = DecisionTreeClassifier(max_depth=1, random_state=42)
    modelo.fit(X_train, y_train, sample_weight=pesos)
    pred = modelo.predict(X_train)

    # Etapa 3 e 4: erro e peso do modelo
    erro = np.sum(pesos * (pred != y_train)) / np.sum(pesos)
    erro = np.clip(erro, 1e-10, 1 - 1e-10)
    alfa = 0.5 * np.log((1 - erro) / erro)

    # Etapa 3 - Ajuste dos pesos dos exemplos
    pesos = pesos * np.exp(-alfa * y_train * (2 * (pred == y_train) - 1))
    pesos = pesos / np.sum(pesos)

    modelos.append(modelo)
    alfas.append(alfa)

    print(f"🔁 Iteração {i+1} | Erro: {erro:.4f} | Alfa: {alfa:.4f}")

🔁 Iteração 1 | Erro: 0.1143 | Alfa: 1.0238
🔁 Iteração 2 | Erro: 0.1400 | Alfa: 0.9078
🔁 Iteração 3 | Erro: 0.1685 | Alfa: 0.7982
🔁 Iteração 4 | Erro: 0.1412 | Alfa: 0.9027
🔁 Iteração 5 | Erro: 0.1797 | Alfa: 0.7593


## 📌 Etapa 5 - Repetição

Repetimos esse processo para `n_estimators` vezes. No código acima, usamos um loop com 5 iterações.

---

## 📌 Etapa 6 - Agregação das previsões

A predição final é feita com uma votação ponderada.

In [11]:
from sklearn.metrics import accuracy_score

# Votação ponderada no conjunto de teste
predicoes = np.zeros_like(y_test, dtype=float)
for modelo, alfa in zip(modelos, alfas):
    pred = modelo.predict(X_test)
    pred = np.where(pred == 1, 1, -1)
    predicoes += alfa * pred

final = np.where(predicoes > 0, 1, 0)
print("🎯 Acurácia final do AdaBoost manual (com dados de teste):", accuracy_score(y_test, final))

🎯 Acurácia final do AdaBoost manual (com dados de teste): 1.0


# ⚡ 4.2. AdaBoost com Scikit-learn (automático)

---



In [6]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split

# Separar treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Criar AdaBoost automatizado
modelo_base = DecisionTreeClassifier(max_depth=1)
adaboost = AdaBoostClassifier(
    estimator=modelo_base,
    n_estimators=5,
    learning_rate=1.0,
    random_state=42
)

# Treinar e avaliar
adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)
print("🤖 Acurácia com AdaBoost (sklearn):", accuracy_score(y_test, y_pred))

🤖 Acurácia com AdaBoost (sklearn): 1.0


# ⚡ Cinco hiperparâmetros importantes no AdaBoost

Abaixo estão os hiperparâmetros mais relevantes no `AdaBoostClassifier` do scikit-learn:

---

| Hiperparâmetro     | Descrição                                                                                 |
|--------------------|--------------------------------------------------------------------------------------------|
| `n_estimators`     | Número de modelos fracos (árvores) a serem treinados sequencialmente.                     |
| `learning_rate`    | Controla o peso dado a cada modelo. Valores menores tornam o aprendizado mais conservador.|
| `estimator`        | Define o modelo fraco usado (ex: `DecisionTreeClassifier` com `max_depth=1`).             |
| `algorithm`        | Escolhe o tipo de AdaBoost: `'SAMME'` (discreto) ou `'SAMME.R'` (real/probabilístico).    |
| `random_state`     | Semente aleatória para garantir reprodutibilidade.                                        |

---

 🧠 Observações adicionais:

- **`n_estimators`**: quanto maior, mais modelos são combinados. Porém, após certo ponto, o ganho de performance pode estabilizar.
- **`learning_rate`**: valores muito altos podem causar overfitting; muito baixos, underfitting. O valor padrão é `1.0`.
- **`estimator`**: normalmente usamos árvores rasas (stumps), mas você pode experimentar modelos mais complexos.
- **`algorithm='SAMME.R'`**: recomendado para classificação multiclasse; usa probabilidades e costuma performar melhor.
- **`random_state`**: garante que os resultados sejam reproduzíveis entre execuções.

---



⚙️ Exemplo com GridSearchCV para ajuste de hiperparâmetros

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    'n_estimators': [5, 10, 50],
    'learning_rate': [0.1, 0.5, 1.0]
}

grid = GridSearchCV(
    estimator=AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1), random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train, y_train)
print("🔍 Melhores parâmetros:", grid.best_params_)
print("✅ Acurácia com GridSearchCV:", accuracy_score(y_test, grid.best_estimator_.predict(X_test)))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
🔍 Melhores parâmetros: {'learning_rate': 1.0, 'n_estimators': 10}
✅ Acurácia com GridSearchCV: 1.0
